Notebook for the diffusion development program

In [1]:
import model.diffusion as gd
diffusion = gd.create_diffusion_model()

/home/alden/miniconda3/envs/pytorch-cuda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



Model created successfully
- Total parameters: 57,314,049
- Trainable parameters: 57,314,049
- Model size: 218.6 MB
- Device: cuda:0
- Model Name: LatentDiffusionModel


In [2]:
diffusion.sample()

DDPM Sampling loop time step: 100%|██████████| 1000/1000 [00:10<00:00, 98.03it/s]


tensor([[[ 1.9967, -0.4015,  0.9483,  ...,  3.1158,  1.2388, -0.6542]],

        [[ 1.8801, -0.6903,  1.3920,  ...,  1.1581,  1.6686,  0.8780]],

        [[ 0.9055,  1.1426,  2.0282,  ...,  1.5317,  0.6035, -0.9308]],

        ...,

        [[ 0.5549, -0.0256,  2.1864,  ...,  0.7039, -0.1407,  1.3528]],

        [[ 0.1941, -0.0676, -0.0688,  ...,  1.6306,  0.3105,  0.6058]],

        [[-1.1207, -0.1519,  2.6668,  ...,  1.0149,  1.4031, -2.0516]]],
       device='cuda:0')

Testing:

In [ ]:
model = create_diffusion_model()
load_diffusion_model(model=model, model_path="saved_models/diffusion/molecule-diffusion-v1.pt", device="cuda")


Model created successfully
- Total parameters: 57,314,049
- Trainable parameters: 57,314,049
- Model size: 218.6 MB
- Device: cuda:0
- Model Name: GaussianDiffusion1D


/tmp/ipykernel_116813/3819632101.py:61: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_path)


Loaded model successfully from saved_models/diffusion/molecule-diffusion-v1.pt
